In [1]:
import pandas as pd

#Recipes and Ingredients: Simple Cookbook
df_recipe_simple = pd.read_csv("data\\recipe.csv")
df_ingredient_simple = pd.read_csv("data\\ingredient.csv")

In [2]:
folder_path = "data\\atlas-query-cookbook"

df_ingredient = pd.read_csv(folder_path + r"\Ingredient", sep='\t')
df_nutrition = pd.read_csv(folder_path + r"\Nutrition", sep='\t')
df_quantity = pd.read_csv(folder_path + r"\Quantity", sep='\t')
df_recipe = pd.read_csv(folder_path + r"\Recipe", sep=',')
df_ingredient = df_ingredient.applymap(lambda x: x.strip() if isinstance(x, str) else x)
df_quantity = df_quantity.applymap(lambda x: x.strip() if isinstance(x, str) else x)

C:\Users\20203717\AppData\Local\Temp\ipykernel_19440\4248548079.py:6: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_ingredient = df_ingredient.applymap(lambda x: x.strip() if isinstance(x, str) else x)
C:\Users\20203717\AppData\Local\Temp\ipykernel_19440\4248548079.py:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_quantity = df_quantity.applymap(lambda x: x.strip() if isinstance(x, str) else x)


In [3]:
df_ingredient = (df_ingredient.drop(columns=['plural'])
                 .rename(columns = {"name": "ingredient_name"}))
new_order = ['ingredient_id', 'ingredient_name', 'category']
df_ingredient = df_ingredient[new_order]
df_ingredient

,ingredient_id,ingredient_name,category
0,1,1% lowfat cottage cheese,dairy
1,6,1% lowfat milk,dairy
2,10,10-inch flour tortilla,Mexican products
3,11,100% bran cereal,cereals
4,12,2% lowfat milk,dairy
...,...,...,...
3342,4641,cooked oatmeal,cereals
3343,4642,instant coffee granules,hot beverages
3344,4643,long grain enriched white rice,grains
3345,4644,frozen grapefruit juice concentrate,frozen fruit


In [4]:
df_quantity_ingredient = pd.merge(df_quantity, df_ingredient, on='ingredient_id')
df_quantity_ingredient["qty"] = (df_quantity_ingredient["min_qty"] + df_quantity_ingredient["max_qty"]) / 2
df_quantity_ingredient = (df_quantity_ingredient
                          .drop(columns = ["quantity_id", "max_qty", "min_qty", "preparation", "optional"]))

new_order = ['recipe_id', 'ingredient_id', 'ingredient_name', 'category', 'unit', 'qty']
df_quantity_ingredient = df_quantity_ingredient[new_order]
df_quantity_ingredient

,recipe_id,ingredient_id,ingredient_name,category,unit,qty
0,214,1613,graham cracker crumbs,cookies/crackers,cup(s),2.00
1,214,3334,sugar,baking products,cup(s),0.25
2,214,2222,margarine or butter,dairy,cup(s),0.50
3,214,2797,raspberry juice,fruit juices,cup(s),0.25
4,214,3567,unflavored gelatin,gelatin,teaspoon(s),3.00
...,...,...,...,...,...,...
5087,1119,289,whole boneless turkey,meat/poultry,pound(s),6.00
5088,1119,3261,Sonoma County chardonnay wine,alcoholic beverages,milliliter(s),750.00
5089,1119,638,chicken broth,soups,cup(s),1.00
5090,1119,1420,fresh rosemary,fresh herbs,teaspoon(s),3.00


In [5]:
def replacer(df, qty_type, multiplier, new_qty='gram(s)'):
    df.loc[df.unit == qty_type, 'qty'] *= multiplier
    df['unit'] = df['unit'].replace({qty_type: new_qty})
    return df

In [6]:
lst_qty = df_quantity.unit.unique()

for qty_type in lst_qty:
    ounce = qty_type.split(" ")[0]
    try:
        ounce_int = int(ounce)
    except:
        continue
    replacer(df_quantity_ingredient, qty_type, ounce_int * 29)
    
replacer(df_quantity_ingredient, 'pound(s)', 455)
replacer(df_quantity_ingredient, 'cup(s)', 240)
replacer(df_quantity_ingredient, 'ounce(s)', 29)
replacer(df_quantity_ingredient, 'fl. ounce(s)', 29)
replacer(df_quantity_ingredient, 'pint(s)', 12*29)
replacer(df_quantity_ingredient, 'drop(s)', 0.05, 'milliliter(s)')
replacer(df_quantity_ingredient, 'stalk(s)', 1, 'whole')
replacer(df_quantity_ingredient, 'head(s)', 1, 'whole')
replacer(df_quantity_ingredient, 'loaf', 1, 'whole')
replacer(df_quantity_ingredient, 'pinch(es)', 0.125, 'teaspoon(s)')
replacer(df_quantity_ingredient, 'dash(es)', 0.125, 'teaspoon(s)')
replacer(df_quantity_ingredient, 'packet(s)', 1, 'package(s)')
replacer(df_quantity_ingredient, 'envelope(s)', 1, 'package(s)')
replacer(df_quantity_ingredient, 'teaspoon(s)', 5)
replacer(df_quantity_ingredient, 'tablespoon(s)', 15)
replacer(df_quantity_ingredient, 'stick(s)', 1, 'piece(s)')
replacer(df_quantity_ingredient, 'slice(s)', 1, 'piece(s)')
replacer(df_quantity_ingredient, 'clove(s)', 1, 'piece(s)')
replacer(df_quantity_ingredient, 'scoop(s)', 1, 'piece(s)')
replacer(df_quantity_ingredient, 'bunch(es)', 1, 'piece(s)')
replacer(df_quantity_ingredient, 'sprig(s)', 1, 'piece(s)')

df_quantity_ingredient.loc[(df_quantity_ingredient.unit == 'quart(s)') & (df_quantity_ingredient.category.isin(['soups', 'beverages'])), 'qty'] *= 946
df_quantity_ingredient.loc[(df_quantity_ingredient.unit == 'quart(s)') & (df_quantity_ingredient.category.isin(['soups', 'beverages'])), 'unit'] = 'gram(s)'
replacer(df_quantity_ingredient, 'quart(s)', 1, 'piece(s)')

df_quantity_ingredient.where(df_quantity_ingredient.unit == 'whole').dropna()   

,recipe_id,ingredient_id,ingredient_name,category,unit,qty
436,274.0,758.0,cloves,spices and seasonings,whole,2.000
443,275.0,758.0,cloves,spices and seasonings,whole,3.000
730,422.0,758.0,cloves,spices and seasonings,whole,6.000
741,424.0,585.0,celery,fresh vegetables,whole,1.000
771,429.0,1814.0,Idaho potato,fresh vegetables,whole,4.000
811,435.0,967.0,cucumber,fresh vegetables,whole,0.875
913,450.0,397.0,California avocado,fresh vegetables,whole,1.000
1026,470.0,2277.0,red cabbage,fresh vegetables,whole,1.000
2156,644.0,1484.0,frozen bread dough,frozen baking products,whole,0.500
2432,664.0,3714.0,whole butterhead lettuce,fresh vegetables,whole,2.000


In [7]:
def replacer2(df, qty_type, ing_name, multiplier, new_qty='gram(s)'):
    df.loc[(df.unit == qty_type) & (df.ingredient_name == ing_name), 'qty'] *= multiplier
    df.loc[(df.unit == qty_type) & (df.ingredient_name == ing_name), 'unit'] = new_qty
    return df

In [8]:
replacer2(df_quantity_ingredient, 'whole', 'cloves', 30)
replacer2(df_quantity_ingredient, 'whole', 'celery', 40)
replacer2(df_quantity_ingredient, 'whole', 'Idaho potato', 200)
replacer2(df_quantity_ingredient, 'whole', 'cucumber', 300)
replacer2(df_quantity_ingredient, 'whole', 'California avocado', 150)
replacer2(df_quantity_ingredient, 'whole', 'red cabbage', 1000)
replacer2(df_quantity_ingredient, 'whole', 'frozen bread dough', 500)
replacer2(df_quantity_ingredient, 'whole', 'whole butterhead lettuce', 400)
replacer2(df_quantity_ingredient, 'whole', 'lettuce', 300)
replacer2(df_quantity_ingredient, 'whole', 'radicchio', 200)
replacer2(df_quantity_ingredient, 'whole', 'corn on the cob', 200)
replacer2(df_quantity_ingredient, 'whole', 'skinless boneless chicken breast', 175)
replacer2(df_quantity_ingredient, 'whole', 'chicken breast', 175)
replacer2(df_quantity_ingredient, 'whole', 'iceberg lettuce', 700)
replacer2(df_quantity_ingredient, 'whole', 'cooked broiler-fryer, boned, skinned', 600)
replacer2(df_quantity_ingredient, 'whole', 'cabbage', 900)
replacer2(df_quantity_ingredient, 'whole', 'cantaloupe', 1500)
replacer2(df_quantity_ingredient, 'whole', 'New Zealand kiwi fruit', 75)
replacer2(df_quantity_ingredient, 'whole', 'green onion', 15)
replacer2(df_quantity_ingredient, 'whole', 'butter lettuce', 150)
replacer2(df_quantity_ingredient, 'whole', 'onion', 150)
replacer2(df_quantity_ingredient, 'whole', 'French bread', 250)
replacer2(df_quantity_ingredient, 'whole', 'Belgian endive', 80)

,recipe_id,ingredient_id,ingredient_name,category,unit,qty
0,214,1613,graham cracker crumbs,cookies/crackers,gram(s),480.0
1,214,3334,sugar,baking products,gram(s),60.0
2,214,2222,margarine or butter,dairy,gram(s),120.0
3,214,2797,raspberry juice,fruit juices,gram(s),60.0
4,214,3567,unflavored gelatin,gelatin,gram(s),15.0
...,...,...,...,...,...,...
5087,1119,289,whole boneless turkey,meat/poultry,gram(s),2730.0
5088,1119,3261,Sonoma County chardonnay wine,alcoholic beverages,milliliter(s),750.0
5089,1119,638,chicken broth,soups,gram(s),240.0
5090,1119,1420,fresh rosemary,fresh herbs,gram(s),15.0


In [9]:
replacer2(df_quantity_ingredient, 'package(s)', 'unflavored gelatin', 7)
replacer2(df_quantity_ingredient, 'package(s)', 'sugar substitute', 50)
replacer2(df_quantity_ingredient, 'package(s)', 'yeast', 7)
replacer2(df_quantity_ingredient, 'package(s)', 'active dry yeast', 7)
replacer2(df_quantity_ingredient, 'package(s)', 'low sodium beef broth', 907)
replacer2(df_quantity_ingredient, 'package(s)', 'dry taco seasoning mix', 28)
replacer2(df_quantity_ingredient, 'package(s)', 'cornbread mix', 240)
replacer2(df_quantity_ingredient, 'package(s)', 'chicken noodle soup mix', 60)
replacer2(df_quantity_ingredient, 'package(s)', 'dry ranch dressing mix', 28)
replacer2(df_quantity_ingredient, 'package(s)', 'animal crackers', 56)

,recipe_id,ingredient_id,ingredient_name,category,unit,qty
0,214,1613,graham cracker crumbs,cookies/crackers,gram(s),480.0
1,214,3334,sugar,baking products,gram(s),60.0
2,214,2222,margarine or butter,dairy,gram(s),120.0
3,214,2797,raspberry juice,fruit juices,gram(s),60.0
4,214,3567,unflavored gelatin,gelatin,gram(s),15.0
...,...,...,...,...,...,...
5087,1119,289,whole boneless turkey,meat/poultry,gram(s),2730.0
5088,1119,3261,Sonoma County chardonnay wine,alcoholic beverages,milliliter(s),750.0
5089,1119,638,chicken broth,soups,gram(s),240.0
5090,1119,1420,fresh rosemary,fresh herbs,gram(s),15.0


In [10]:
replacer2(df_quantity_ingredient, 'piece(s)', 'cinnamon', 1)
replacer2(df_quantity_ingredient, 'piece(s)', 'frozen vanilla yogurt', 133)
replacer2(df_quantity_ingredient, 'piece(s)', 'fresh strawberry', 750)
replacer2(df_quantity_ingredient, 'piece(s)', 'Parchment paper or foil', 0)
replacer2(df_quantity_ingredient, 'piece(s)', 'Italian bread', 25)
replacer2(df_quantity_ingredient, 'piece(s)', 'cilantro', 5)
replacer2(df_quantity_ingredient, 'piece(s)', 'garlic clove', 5)
replacer2(df_quantity_ingredient, 'piece(s)', 'fresh parsley', 1)
replacer2(df_quantity_ingredient, 'piece(s)', 'fresh spinach leaves', 350)
replacer2(df_quantity_ingredient, 'piece(s)', 'fresh cilantro', 5)
replacer2(df_quantity_ingredient, 'piece(s)', 'watercress', 125)
replacer2(df_quantity_ingredient, 'piece(s)', 'large garlic clove', 7)
replacer2(df_quantity_ingredient, 'piece(s)', 'bacon', 15)
replacer2(df_quantity_ingredient, 'piece(s)', 'tomato', 20)
replacer2(df_quantity_ingredient, 'piece(s)', 'garlic', 5)
replacer2(df_quantity_ingredient, 'piece(s)', 'lasagne', 200)
replacer2(df_quantity_ingredient, 'piece(s)', 'tortellini deli salad', 750)
replacer2(df_quantity_ingredient, 'piece(s)', 'eggplant', 500)
replacer2(df_quantity_ingredient, 'piece(s)', 'apple', 750)
replacer2(df_quantity_ingredient, 'piece(s)', 'spinach', 200)
replacer2(df_quantity_ingredient, 'piece(s)', 'alfalfa sprouts', 100)
replacer2(df_quantity_ingredient, 'piece(s)', 'red bell pepper', 40)
replacer2(df_quantity_ingredient, 'piece(s)', 'Swiss chard', 350)
replacer2(df_quantity_ingredient, 'piece(s)', 'whole wheat bread', 28)
replacer2(df_quantity_ingredient, 'piece(s)', 'white bread', 25)
replacer2(df_quantity_ingredient, 'piece(s)', 'rye bread', 28)
replacer2(df_quantity_ingredient, 'piece(s)', 'pumpernickel bread', 28)
replacer2(df_quantity_ingredient, 'piece(s)', 'bologna', 28)
replacer2(df_quantity_ingredient, 'piece(s)', 'beef lunchmeat', 28)
replacer2(df_quantity_ingredient, 'piece(s)', 'green onion tops', 120)
replacer2(df_quantity_ingredient, 'piece(s)', 'mustard green', 400)
replacer2(df_quantity_ingredient, 'piece(s)', 'parsley', 1)
replacer2(df_quantity_ingredient, 'piece(s)', 'ice cream', 113)
replacer2(df_quantity_ingredient, 'piece(s)', 'bread', 25)

,recipe_id,ingredient_id,ingredient_name,category,unit,qty
0,214,1613,graham cracker crumbs,cookies/crackers,gram(s),480.0
1,214,3334,sugar,baking products,gram(s),60.0
2,214,2222,margarine or butter,dairy,gram(s),120.0
3,214,2797,raspberry juice,fruit juices,gram(s),60.0
4,214,3567,unflavored gelatin,gelatin,gram(s),15.0
...,...,...,...,...,...,...
5087,1119,289,whole boneless turkey,meat/poultry,gram(s),2730.0
5088,1119,3261,Sonoma County chardonnay wine,alcoholic beverages,milliliter(s),750.0
5089,1119,638,chicken broth,soups,gram(s),240.0
5090,1119,1420,fresh rosemary,fresh herbs,gram(s),15.0


In [11]:
df_recipe_nutrition = pd.merge(df_recipe, df_nutrition, on='recipe_id')
df_recipe_nutrition = (df_recipe_nutrition.drop(columns=["yield_unit", "yield_unit", "prep_min", "cook_min", "stnd_min",
                                                         "source", "intro", "directions"]))

df_quantity_agg = (df_quantity[["recipe_id", "ingredient_id"]]
                    .groupby("recipe_id")["ingredient_id"]
                    .agg(list)
                    .reset_index()
                    .rename(columns = {"ingredient_id": "ingredients"}))
df_quantity_agg["ingredients"] = df_quantity_agg["ingredients"].apply(lambda x: sorted(x))

df_total_recipe = pd.merge(df_recipe_nutrition, df_quantity_agg, on = "recipe_id")

#Dividing the nutrition values by the servings
num_col = df_total_recipe.columns.difference(['recipe_id', 'title', 'subtitle', 'servings', 'ingredients'])
df_total_recipe[num_col] = df_total_recipe[num_col].div(df_total_recipe['servings'], axis=0)

dct_servings = {}
for i, v in enumerate(df_total_recipe.recipe_id):
    dct_servings[v] = df_total_recipe.servings[i]
    
df_total_recipe.drop(columns=['servings'], inplace=True)
df_total_recipe

,recipe_id,title,subtitle,protein,carbo,alcohol,total_fat,sat_fat,cholestrl,sodium,iron,vitamin_c,vitamin_a,fiber,pcnt_cal_carb,pcnt_cal_fat,pcnt_cal_prot,calories,ingredients
0,214,Raspberry Chiffon Pie,NaN,0.547000,4.129000,0.0000,1.153000,0.221000,0.139000,26.07800,0.081000,0.889000,58.620000,0.087000,5.680000,3.568000,0.7530,29.079000,"[924, 1414, 1613, 2196, 2222, 2797, 3334, 3334..."
1,215,Apricot Yogurt Parfaits,NaN,1.425000,5.937500,0.4825,0.270000,0.145000,0.870000,11.54250,0.142500,3.255000,684.560000,0.155000,16.845000,1.722500,4.0425,35.252500,"[71, 76, 1286, 2019, 2196, 3567]"
2,216,Fresh Apricot Bavarian,NaN,0.612500,3.360000,0.0000,0.137500,0.072500,0.432500,5.22375,0.046250,0.766250,190.137500,0.042500,9.806250,0.905000,1.7875,17.132500,"[1286, 2196, 3334, 3567, 3649]"
3,217,Fresh Peaches,with Banana Cream Whip,0.442500,4.542500,0.0000,0.052500,0.015000,0.000000,3.50250,0.047500,2.197500,119.522500,0.172500,22.245000,0.587500,2.1675,20.425000,"[792, 1402, 2019]"
4,218,Canned Cherry Crisp,NaN,0.230000,6.105000,0.0000,0.911667,0.576667,1.726667,8.37000,0.110000,0.026667,38.193333,0.175000,12.135000,4.076667,0.4550,33.538333,"[30, 34, 351, 374, 614, 770, 1684, 2948]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
838,1410,No-Bake Chocolate Cheesecake,NaN,0.541667,2.400833,0.0000,3.770000,2.040833,8.648333,14.42250,0.113333,0.049167,106.975000,0.046667,1.750833,6.186667,0.3950,45.703333,"[35, 249, 374, 374, 496, 782, 792, 921, 1755, ..."
839,1411,Mexican Cocoa Torte,NaN,0.324000,3.867000,0.0000,3.589000,1.467000,6.537000,23.87500,0.107000,0.030000,70.114000,0.009000,3.153000,6.583000,0.2640,49.067000,"[738, 1755, 1767, 2627, 3118, 3329, 3334]"
840,1426,-Pancakes-,NaN,1.572500,7.310000,0.0000,1.625000,0.445000,14.835000,61.18250,0.142500,0.107500,34.370000,0.020000,14.575000,7.287500,3.1375,50.157500,"[30, 126, 2327, 3021, 3334, 3595, 3618]"
841,1427,-Waffles-,NaN,0.821250,3.767500,0.0000,1.717500,0.356250,7.547500,21.73500,0.073750,0.063750,18.451250,0.012500,5.571250,5.713750,1.2150,33.812500,"[30, 125, 1696, 2327, 3021, 3334, 3618]"


In [12]:
for i, v in enumerate(df_quantity_ingredient.recipe_id):
    df_quantity_ingredient.loc[i, 'qty'] /= dct_servings[v]

In [13]:
def recipe_ingredients(df, recipe_id):
    return df[df["recipe_id"] == recipe_id]

recipe_ingredients(df_quantity_ingredient, 430)
recipe_ingredients(df_quantity_ingredient, 430)

,recipe_id,ingredient_id,ingredient_name,category,unit,qty
780,430,1440,fresh strawberry,fresh fruit,gram(s),125.000000
781,430,2454,orange juice,frozen fruit,gram(s),40.000000
782,430,2776,quick cooking tapioca,cereals,gram(s),1.666667
783,430,1684,ground cinnamon,spices and seasonings,gram(s),0.833333
784,430,386,buttermilk,dairy,gram(s),40.000000
785,430,3334,sugar,baking products,gram(s),30.000000
786,430,2019,lemon juice,fruit juices,gram(s),3.750000
787,430,3813,plain yogurt,dairy,gram(s),40.000000


In [14]:
df_recipe[df_recipe.recipe_id == 751]

,recipe_id,title,subtitle,servings,yield_unit,prep_min,cook_min,stnd_min,source,intro,directions
474,751,Applesauce,NaN,4,NaN,20,0,0,"National Heart, Lung and Blood Institute","When fresh apples are at their peak, prepare ...","In medium saucepan, combine all ingredients e..."


In [15]:
# df_quantity_ingredient.to_csv("data_cleaned\\quantity_ingredient_final.csv", index=False)